<H2>Grillage Model from Dxf import</H2>

We'll use the ezdxf python package to raed the dxf -> https://ezdxf.readthedocs.io/en/stable/tutorials/getting_data.html

Note that LUSAS already includes an import from DXF function which is more robust and efficient then this approach. This example shows however that creation and assignmnet of model attribute can be automated at the same time as model geometry creation and how this maybe extended further by including metadata in the dxf that could be used for this process

Import the libraries

In [17]:
import ezdxf 
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.0")
db = lusas.getDatabase()

Options/inputs

In [18]:
dxf_path = "Grillage.dxf" # File path to the dxf file.
create_new_model = True   # Option to create a new model. Warning this will close the existing model without saving 
model_file_name = "Grillage From DXF"

# Geometric Sections
precast_libray = "UK Sections"
slab_thickness = 0.25

# Define sections as a dictionary of name followed by a tuple of dimensions

# Transverse slab (breadth, depth)
transverse_slab_dictionary = { "Transverse Slab":(2.0, slab_thickness) }

# Precast beams with top slabs, ("Beam Type", "Beam Size", slab breadth, slab depth)
precast_dictionary = { "Edge Beams":("Precast TYE Beams", "TYE11", 2.0, slab_thickness), "Inner Beams":("Precast TY Beams", "TY11", 2.0, slab_thickness) }

# Rectangular sections (breadth, depth)
sections_dictionary = { "Diaphrams":(0.75, 1.25) }



Create the new model if required

In [19]:
if create_new_model:
    db = lusas.newDatabase("Structural", f"{model_file_name}.mdl")
    db.setAnalysisCategory("2D Grillage/Plate")
    db.setModelUnits("kN,m,t,s,C")
    db.setTimescaleUnits("Seconds")

    lusas.fileOpen("%Scripts%\Treeview\grillage.lsc") # Use the grillage template script to set up default attributes

We'll read the dxf and create the equivalent geometry in LUSAS Modeller.

In [20]:
# Read the dxf file
doc = ezdxf.readfile(dxf_path)
# Get the modelspace
msp = doc.modelspace()

Loop through the dxf "LINE"s and create the equivalent lines in Modeller. Note that the grillage here is defined as simple straight lines. More complex geometries can be considered with polylines

In [ ]:
for e in msp:
    if e.dxftype() == "LINE":
        layer = e.dxf.layer  # layer of the entity as string

        if not db.exists("Group", layer):
            group = db.createEmptyGroup(layer)
        else:
            group = db.getGroupByName(layer)

        # Points at the start and end of the line
        p1 = e.dxf.start
        p2 = e.dxf.end

        # Create Line in LUSAS
        # Reference to the geometryData object
        geomData = lusas.geometryData()
        geomData.setAllDefaults()               # Reset the inputs
        geomData.addCoords(p1.x, p1.y, p1.z)    # Set the coordinates of the first point
        geomData.addCoords(p2.x, p2.y, p2.z)    # Set the coordinates of the second point
        geomData.setLowerOrderGeometryType("coordinates")
        geomData.setCreateMethod("straight")
        # Create the line and add it to the group
        line = win32.CastTo(db.createLine(geomData).getObject("Line"), "IFLine")
        group.add(line) 


<H3>Create and assign Geometric attributes</H3>

We'll use the "Bridge Deck Grillage" attributes for this

In [22]:
# First the transverse slab sections. The dimensions tuple contains a breadth and thickness
for key in transverse_slab_dictionary.keys():
    dims = transverse_slab_dictionary[key]
    attr = db.createGeometricLine(key)
    # attr.setValue("elementType", "3D Thick Beam")
    attr.setBridgeDeckSection("RC slab", "Solid")
    attr.setSlabTreatment("Uncracked")
    attr.setTorsionConstant("Grillage slab")
    attr.addBridgeDeckSlabA(dims[0], dims[1])
    if db.exists("Group", key):
        attr.assignTo("Group", key)

# Precast beams with deck slab
for key in precast_dictionary.keys():
    dims = precast_dictionary[key]
    attr = db.createGeometricLine(key)
    # attr.setValue("elementType", "3D Thick Beam")
    attr.setBridgeDeckSection("Girder with slab", "Composite")
    attr.setSlabTreatment("Uncracked - concrete only")
    attr.setTorsionConstant("Sum slab and girder", "None (use St Venant)", 0.0)
    attr.setFromLibrary(precast_libray, dims[0], dims[1], 0, 0, 0)
    attr.addBridgeDeckSlabA(dims[2], dims[3])
    if db.exists("Group", key):
        attr.assignTo("Group", key)


# Rectangular sections
for key in sections_dictionary.keys():
    # First the section definition
    attr = db.createParametricSection(key)
    attr.setType("Rectangular Solid")
    names = ["B", "D"]
    values = sections_dictionary[key]
    attr.setDimensions(names, values)
    # Then the geometric attribute which references the section definition
    attr = db.createGeometricLine(key)
    attr.setFromLibrary("Utilities", "", key, 0, 0, 0)
    attr.setValue("Epw", values[0], 0)
    if db.exists("Group", key):
        attr.assignTo("Group", key)


Assign the mesh attribute

In [23]:

if db.existsAttribute("Mesh", "Grillage element div=1"):
    attr = db.getAttribute("Mesh", "Grillage element div=1")
    attr.assignToAll(lusas.assign().setAllDefaults())
    db.updateMesh() # Calculate the mesh so we can visualise assignments


Set the display

In [24]:

lusas.view().geometry().colourByParentGroup(True)
lusas.view().attributes().setVisible(True)
lusas.view().attributes().setDrawStyle("Geometric", "Arrows")
lusas.view().attributes().visualiseAll("Geometric")
lusas.view().attributes().visualiseAllTransparent("Geometric")

